# Mini-Project 2 : Customer Feedback Analysis Project

1. Importing libraries

In [ ]:
import pandas as pd
import openai
from openai import OpenAI
import os
# import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns

2. Loading the dataset and preprocessing

In [ ]:
reviews = pd.read_json('Musical_Instruments_5.json', lines=True)
data = reviews.copy()
data.head()

In [ ]:
data.info()

In [ ]:
data.isnull().sum()

In [ ]:
data.tail(30)

In [ ]:
# fill NAN values with 'Unknown'
data.fillna('Unknown', inplace=True)
data.isnull().sum()

In [ ]:
data['overall'].unique() # no outliers found

2. Natural Language Analysis with ChatGPT:

In [ ]:
# set my api key
openai.api_key = os.getenv('OPENAI_API_KEY')
client = openai.OpenAI()

In [ ]:
# Function to analyze each review
def analyze_review(review_text):
    messages = [
        {"role": "system", "content": "You are an assistant that analyzes customer reviews."},
        {"role": "user", "content": f"Analyze the following review and categorize the feedback into themes and sentiments: {review_text}"}
    ]
    
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        max_tokens=150,
        n=1,
        stop=None,
        temperature=0.5,
    )
    
    return response.choices[0].message.content.strip()

In [ ]:
data['Analysis'] = data['reviewText'].apply(analyze_review)
print(data.head(30))